In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import tensorflow as tf

from tensorflow.keras.models import Model, load_model
from tqdm import tqdm


2021-10-12 14:19:21.533460: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2021-10-12 14:19:21.533630: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
img = np.load('/kaggle/input/rsna-miccai-voxel-256-dataset/voxel/train/00000/FLAIR.npy')

In [3]:
class DataLoader(tf.keras.utils.Sequence):
    def __init__(self,base_dir='/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification/test',\
                mods=['FLAIR']):
        self.batch_size = 1 
        self.base_dir = base_dir
        self.pat_ids = sorted(glob.glob(os.path.join(base_dir, '*')))
        self.modalities = mods
        print('PAT IDS:',len(self.pat_ids),' | Modalities:',self.modalities)
    
    def __getitem__(self,index):
        batch_patids = self.pat_ids[index:index+self.batch_size]
        all_images = {}
        for K in self.modalities:
            all_images[K] = {'images':[],'ids':[]}
        for patid in batch_patids:
            for MOD in all_images.keys():
                all_images[MOD]['images'].append(np.load(os.path.join(patid, MOD+'.npy')))
                all_images[MOD]['ids'].append(patid.replace('\\','/').split('/')[-1])
        return all_images
   
    def __len__(self):
        return int(len(self.pat_ids)/self.batch_size)

In [4]:
class StageOne:
    def __init__(self,modelpath='',height=256,width=256,score_min_thresh=0.5):
        self.model = load_model(modelpath)
        self.score_min_thresh = score_min_thresh
        self.height = height
        self.width = width
        self.offset_perc = 0.1
    
    def infer(self,image_batch,filter_batchsize=16):
        filtered_images = {}
        for K in image_batch.keys():
            filtered_images[K] = {'images':[],'ids':[]}
            for imagesbatch, patid in zip(image_batch[K]['images'],image_batch[K]['ids']):
                div,mod = divmod(len(imagesbatch),filter_batchsize)
                if mod!=0:
                    div+=1
                dset = tqdm(range(0,len(imagesbatch),filter_batchsize),total=div,position=0, leave=True)
                dset.set_description(f'{patid}|Filtering')
                filtered_batch_images = []
                for i in dset:
                    org_batchimgs = imagesbatch[i:i+filter_batchsize]
                    batchimgs = org_batchimgs
                    batchimgs = np.array([cv2.resize(img,(self.width,self.height))/255. for img in batchimgs])
                    out = self.model.predict(batchimgs)
                    maxindexes = np.argmax(out,axis=1)
                    for j in range(len(maxindexes)):
                        if maxindexes[j] == 1 and out[j][maxindexes[j]] >= self.score_min_thresh:
                            filtered_batch_images.append(org_batchimgs[j])
                            
                if len(filtered_batch_images)==0:
                    offset = math.ceil(len(imagesbatch)*self.offset_perc)
                    filtered_batch_images = imagesbatch[offset:-offset]
                filtered_images[K]['images'].append(filtered_batch_images)
                filtered_images[K]['ids'].append(patid)
                filtered_batch_images = None
                dset = None
                
            return filtered_images
              

In [5]:
mods = ['FLAIR']
generator = DataLoader(base_dir='/kaggle/input/rsna-miccai-voxel-256-dataset/voxel/train/')
stage_one = StageOne(modelpath='/kaggle/input/models/FINAL_MODELALL_acc0.9825_ep26.h5')

PAT IDS: 585  | Modalities: ['FLAIR']


2021-10-12 14:19:27.618336: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-10-12 14:19:27.621209: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2021-10-12 14:19:27.621239: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-10-12 14:19:27.621264: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (87ca557a071d): /proc/driver/nvidia/version does not exist
2021-10-12 14:19:27.621519: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operation

In [7]:
dset = tqdm(enumerate(generator),total=len(generator),position=0, leave=True)
dset.set_description('Loading_test')
all_results = {}
for i,sample in dset:
    #if i > 2:
    #    break
    filtered_images = stage_one.infer(sample)
    print(np.array(filtered_images['FLAIR']['images']).shape)

00000|Filtering: 100%|██████████| 16/16 [00:30<00:00,  1.89s/it]
Loading_test:   0%|          | 1/585 [00:30<4:53:44, 30.18s/it]

(1, 68, 256, 256)


00002|Filtering: 100%|██████████| 16/16 [00:30<00:00,  1.89s/it]
Loading_test:   0%|          | 2/585 [01:01<4:57:11, 30.59s/it]

(1, 113, 256, 256)


00003|Filtering: 100%|██████████| 16/16 [00:30<00:00,  1.88s/it]
Loading_test:   1%|          | 3/585 [01:31<4:57:25, 30.66s/it]

...

(1, 132, 256, 256)


00087|Filtering: 100%|██████████| 16/16 [00:30<00:00,  1.88s/it]
Loading_test:   9%|▉         | 55/585 [28:23<4:42:07, 31.94s/it]

(1, 77, 256, 256)


00088|Filtering:  56%|█████▋    | 9/16 [00:18<00:14,  2.07s/it]
Loading_test:   9%|▉         | 55/585 [28:42<4:36:41, 31.32s/it]


KeyboardInterrupt: 

In [ ]:
for i in filtered_images['FLAIR']['images'][0]:
    plt.imshow(i,cmap='gray')
    plt.show()

In [ ]:
sample['FLAIR']['images'][0].shape